# UAM Theory Unit Tests (β_dev, w(z), E²)

**Scope:** deterministic smoketests for core cosmology primitives in `GoldenMaster/theory/uam_model.py`.
- β_dev(w) even symmetry and small-angle limit
- w(z) solver stability on z∈[0,2.3]
- E²(z) positivity and E²(0)=1


In [ ]:
try:
    import numpy as np
    from GoldenMaster.theory.uam_model import beta_dev, solve_w_of_z, E2_uam
    print('[imports] ok')
except Exception as e:
    print('[imports] FAILED:', e)
    raise


In [ ]:
import numpy as np

ok = True

def log(msg):
    print(msg)

# --- β_dev: small-angle and even symmetry ---
w_small = np.array([1e-8, 1e-6, 1e-4])
beta = beta_dev(w_small)
beta_series = (w_small**2)/24.0
rel_err = np.abs(beta - beta_series)/np.maximum(beta_series, 1e-30)
if np.all(rel_err < 1e-3):
    log(f"[beta_dev] small-angle OK: max relerr={rel_err.max():.2e}")
else:
    ok = False; log(f"[beta_dev] small-angle FAIL: max relerr={rel_err.max():.2e}")

w_test = np.linspace(-1.2, 1.2, 501)
sym = np.max(np.abs(beta_dev(w_test) - beta_dev(-w_test)))
if sym < 1e-10:
    log(f"[beta_dev] even symmetry OK: max |Δ|={sym:.2e}")
else:
    ok = False; log(f"[beta_dev] even symmetry FAIL: max |Δ|={sym:.2e}")

# --- w(z): solver stability & rough monotonicity ---
z_grid = np.linspace(0.0, 2.3, 300)
w = solve_w_of_z(z_grid)
if np.all(np.isfinite(w)):
    log('[w(z)] finite OK')
else:
    ok = False; log('[w(z)] FINITE FAIL (non-finite values)')

dw = np.diff(w)
neg_jumps = np.sum(dw < -1e-4)
if neg_jumps == 0:
    log('[w(z)] monotonic non-decreasing OK (within tol)')
else:
    ok = False; log(f"[w(z)] MONOTONIC FAIL: {neg_jumps} negative jumps < -1e-4")

# --- E²(z): positivity & normalization ---
E2 = E2_uam(z_grid)
emin = E2.min()
if np.all(np.isfinite(E2)) and emin > 0:
    log(f"[E2] positivity OK: min(E2)={emin:.6g}")
else:
    ok = False; log(f"[E2] POSITIVITY FAIL: min(E2)={emin:.6g}")

E2_0 = float(E2_uam(0.0))
if abs(E2_0 - 1.0) < 1e-10:
    log(f"[E2] normalization at z=0 OK: E2(0)={E2_0:.12f}")
else:
    ok = False; log(f"[E2] NORMALIZATION FAIL: E2(0)={E2_0:.12f}")

log('[summary] PASS' if ok else '[summary] FAIL')
